# QDrant getting started

> Run first from a terminal

In [1]:
# docker pull qdrant/qdrant
# docker run -p 6333:6333 qdrant/qdrant

In [2]:
!docker logs qdrant --tail 20

           _                 _    
  __ _  __| |_ __ __ _ _ __ | |_  
 / _` |/ _` | '__/ _` | '_ \| __| 
| (_| | (_| | | | (_| | | | | |_  
 \__, |\__,_|_|  \__,_|_| |_|\__| 
    |_|                           

Access web UI at http://localhost:6333/dashboard

[2023-08-18T11:41:15.280Z INFO  storage::content_manager::consensus::persistent] Initializing new raft state at ./storage/raft_state.json
[2023-08-18T11:41:15.334Z INFO  qdrant] Distributed mode disabled
[2023-08-18T11:41:15.334Z INFO  qdrant] Telemetry reporting enabled, id: 03da7b0c-3192-441d-9950-744ffd722f6c
[2023-08-18T11:41:15.340Z INFO  qdrant::tonic] Qdrant gRPC listening on 6334
[2023-08-18T11:41:15.340Z INFO  qdrant::tonic] TLS disabled for gRPC API
[2023-08-18T11:41:15.342Z INFO  qdrant::actix] TLS disabled for REST API
[2023-08-18T11:41:15.342Z INFO  qdrant::actix] Qdrant HTTP listening on 6333
[2023-08-18T11:41:15.342Z INFO  actix_server::builder] Starting 15 workers
[2023-08-18T11:41:15.342Z INFO  actix_server::serv

In [3]:
pip install qdrant-client

Note: you may need to restart the kernel to use updated packages.


In [4]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import numpy as np
from faker import Faker

In [5]:
client = QdrantClient(host="localhost", port=6333)
client

In [6]:
my_collection="first collection"
client.create_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=100, distance=models.Distance.COSINE)
)

True

In [7]:
data = np.random.uniform(low=-1.0, high=1.0, size=(1_000, 100))
index = list(range(len(data)))

In [8]:
data[:2,:10]

array([[-0.80888272, -0.85981389, -0.77255427,  0.28150384, -0.54741752,
         0.58079336,  0.39604837, -0.56025013, -0.52022725, -0.63509455],
       [ 0.04350789, -0.50781614, -0.65557692,  0.41984314, -0.61817814,
        -0.94368377, -0.69431677,  0.37995716,  0.84685145, -0.2215611 ]])

In [9]:
index[-10:]

[990, 991, 992, 993, 994, 995, 996, 997, 998, 999]

In [10]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist()
    )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [11]:
client.retrieve(
    collection_name=my_collection,
    ids=[10, 14, 500],
    # with_vectors=True
)

[Record(id=500, payload={}, vector=None),
 Record(id=10, payload={}, vector=None),
 Record(id=14, payload={}, vector=None)]

In [12]:
fake_something= Faker()
fake_something.name(), fake_something.address()

('Taylor Carlson', '79170 Sara Islands\nRubenchester, ME 35115')

In [13]:
payload = []
for i in range(1_000):
    payload.append(
        {
        "artist": fake_something.name(),
        "song": " ".join(fake_something.words()),
        "url_song": fake_something.url(),
        "year": fake_something.year(),
        "country": fake_something.country()
        }
    )
payload[:3]

[{'artist': 'Trevor Boyd',
  'song': 'mention edge husband',
  'url_song': 'http://stokes.com/',
  'year': '1971',
  'country': 'Maldives'},
 {'artist': 'Nathaniel Simmons',
  'song': 'poor entire down',
  'url_song': 'https://www.anderson.net/',
  'year': '1993',
  'country': 'Bahrain'},
 {'artist': 'Kathleen King',
  'song': 'nothing her individual',
  'url_song': 'https://www.harrison.com/',
  'year': '1993',
  'country': 'Swaziland'}]

In [14]:
client.upsert(
    collection_name = my_collection,
    points = models.Batch(
        ids = index,
        vectors=data.tolist(),
        payloads=payload
    )
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

### Semantic Search

In [15]:
living_la_vida_loca = np.random.uniform(low=-1.0, high=1.0, size=(100)).tolist()
living_la_vida_loca[:5]

[-0.29420670629043455,
 -0.050257055327542766,
 0.22891916963552572,
 -0.2760902389581992,
 -0.8374864545947605]

In [16]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    limit=10
)

[ScoredPoint(id=12, version=1, score=0.4059105, payload={'artist': 'Randall Elliott', 'country': 'Reunion', 'song': 'season choose woman', 'url_song': 'https://www.atkins.com/', 'year': '1973'}, vector=None),
 ScoredPoint(id=996, version=1, score=0.28662387, payload={'artist': 'Donald Zavala', 'country': 'Congo', 'song': 'factor mouth decision', 'url_song': 'https://www.walker-gallegos.com/', 'year': '1977'}, vector=None),
 ScoredPoint(id=349, version=1, score=0.26726472, payload={'artist': 'William Jones', 'country': 'Brunei Darussalam', 'song': 'understand history everything', 'url_song': 'http://www.ali.net/', 'year': '1990'}, vector=None),
 ScoredPoint(id=118, version=1, score=0.260951, payload={'artist': 'Tiffany Ryan', 'country': 'Uruguay', 'song': 'form education feeling', 'url_song': 'http://www.sandoval.com/', 'year': '1990'}, vector=None),
 ScoredPoint(id=993, version=1, score=0.25718462, payload={'artist': 'Chelsea Nelson', 'country': 'British Indian Ocean Territory (Chagos 

In [17]:
aussie_songs = models.Filter(
    must=[
        models.FieldCondition(
            key="country", match=models.MatchValue(value="Australia")
        )
    ]
)
aussie_songs

Filter(should=None, must=[FieldCondition(key='country', match=MatchValue(value='Australia'), range=None, geo_bounding_box=None, geo_radius=None, values_count=None)], must_not=None)

In [18]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    query_filter=aussie_songs,
    limit=10
)

[ScoredPoint(id=412, version=1, score=0.033074193, payload={'artist': 'Ms. Melissa Berry', 'country': 'Australia', 'song': 'trouble candidate significant', 'url_song': 'http://clark.com/', 'year': '1979'}, vector=None),
 ScoredPoint(id=323, version=1, score=-0.015690451, payload={'artist': 'Alicia Martin', 'country': 'Australia', 'song': 'arrive discussion commercial', 'url_song': 'http://williams.info/', 'year': '2010'}, vector=None),
 ScoredPoint(id=143, version=1, score=-0.11761058, payload={'artist': 'Carol Flores', 'country': 'Australia', 'song': 'sign sense start', 'url_song': 'https://anderson.com/', 'year': '2011'}, vector=None),
 ScoredPoint(id=438, version=1, score=-0.20966032, payload={'artist': 'Matthew Lynn', 'country': 'Australia', 'song': 'a hope year', 'url_song': 'https://www.henson.com/', 'year': '2016'}, vector=None)]

## Recommendation

In [19]:
client.recommend(
    collection_name=my_collection, 
    query_vector=living_la_vida_loca,
    positive=[17],
    limit=5
)

AssertionError: Unknown arguments: ['query_vector']